In [1]:
import numpy as np
import pandas as pd
import math
import random
import csv

import pickle

from params import *
from functions import *

In [2]:
f = open('results/results_online_SHR_ROBUST.txt', 'w+')

for SEED in range(0,10):

    set_seed(SEED)
    data_incoming = pd.read_pickle(f'data/data_incoming_{SEED}.pickle')
    lambda_h = pickle.load(open(f'results/results_offline_SHR_ROBUST_{SEED}.pickle','rb'))

    delta_objective_function_true_total = 0
    
    # track the remaining budget
    remaining_budget = B
    
    n = 0

    while True:
        
        if (n >= len(data_incoming)):
            break

        fare = data_incoming.loc[n, 'fare']
        compensation = data_incoming.loc[n, 'compensation']
        
        if (fare*passenger_incentive_list[1] > remaining_budget and compensation*driver_incentive_list[1] > remaining_budget):
            break
        
        l_best = -np.inf
        i_best = 0
        j_best = 0

        for i in range(0, len(passenger_incentive_list)):
            for j in range(0, len(driver_incentive_list)):
                
                pred_prob = data_incoming.loc[n, f'pred_prob_{i}_{j}']
                    
                if (fare*passenger_incentive_list[i]+compensation*driver_incentive_list[j] <= remaining_budget and fare*(1-passenger_incentive_list[i]) > compensation*(1+driver_incentive_list[j])):
                    l_current = (pred_prob-r) - lambda_h*(pred_prob+r)*(fare*passenger_incentive_list[i]+compensation*driver_incentive_list[j])
                    if (l_current > l_best):
                        l_best = l_current
                        i_best = i
                        j_best = j
            
        delta_objective_function_true_total += data_incoming.loc[n, f'true_prob_{i_best}_{j_best}'] - data_incoming.loc[n, f'true_prob_0_0']
        remaining_budget -= data_incoming.loc[n, f'true_prob_{i_best}_{j_best}'] * (fare*passenger_incentive_list[i_best]+compensation*driver_incentive_list[j_best])
        
        n += 1
    
    f.write(f'{SEED}, SHR-ROBUST, {delta_objective_function_true_total}, {n}\n')
    
f.close()